In [13]:
import pandas as pd
l = 0
d = {}
with open("data/pap-ned.txt") as fileobject:
    for line in fileobject:
        l += 1
        if "=" not in line:
            continue
        if l >= 109:
            word, definition = line.split("=", 1)
            word = word.strip()[1:]
            definition = definition.strip()
            d[word] = definition
df = pd.DataFrame.from_dict(d, orient="index", columns=["definition"])
df.index.name = "word"
print(df)

                                                    definition
word                                                          
'na          geven (v. <duna>). <'na m'é!> - Geef hier! <Di...
a            [znw] A, a; <di a te z.> - van a tot z. <Esun ...
AAA                                    Aruba Airport Authority
abandoná     opgeven, verlaten, in de steek laten (zie: <ba...
abarká                                     omvatten; inhouden.
...                                                        ...
zundra                         opspelen; schelden; uitschelden
zundrá       uitkaffering; uitbrander; scheldpartij. <Duna ...
zundramento                                           gescheld
zür                                                       zuur
zürdeg                                    zuurdeeg; zuurdesem.

[8318 rows x 1 columns]
